# Data Management and Plotting


## Background

In this project, we work towards a replication of Cunha, Heckman and Schennach (CHS), Econometrica, 2010. We will clean data from the NLSY and transform it into the format needed later.

We will focus on the Behavior Problem Index that is used to measure non-cognitive skills. This index has the subscales antisocial behavior, anxiety, dependence, headstrong, hyperactive and peer problems. Here is an [overview](https://www.nlsinfo.org/content/cohorts/nlsy79-children/other-documentation/codebook-supplement/appendix-d-behavior-proble-0).

The repository contains a directory called `original_data`, with four files in it:

- `BEHAVIOR_PROBLEMS_INDEX.dta`: Contains the main data we work with. It is in wide format and the variable names are not informative. Moreover, the names do not contain the survey year in which the question was asked.
- `bpi_variable_info.csv`: Contains information that help us to decompose the main dataset into datasets for each year and to rename the variables such that the same questions get the same name across periods. 
- `BEHAVIOR_PROBLEMS_INDEX.cdb`: The codebook of the dataset. 
- `chs_data.dta`: The data file used in the original paper by Cunha Heckman and Schennach. 

## Steps we performed


1. We loaded the chs_data. Extract a list of ``childid``s that contains all values that variable takes in the chs_data, but no duplicates. Also, we discarded all observations in which `year` is not in 
```python
    list(range(1986, 2011, 2))
```

2. We cleaned and transformed the bpi dataset:
    - Load the bpi dataset
    - Use the list of `childid`s to only keep observations that are present in the chs_data. The column with child identifiers is called `C0000100` in the bpi dataset.
    - Replace all negative numbers by ``pd.np.nan``
    - Create a dictionary where the keys are the survey years (we only need the years 1986 - 2010) and the values are DataFrames with the bpi data of that year. In these DataFrames:
        - All variables have readable names from `bpi_variable_info.csv`
        - We discarded all variables that are not present in `bpi_variable_info.csv`
        - We added a column called `year`, that indicates the survey year
        

3. We used the results from the previous steps to generate a new bpi dataset in long format. We saved this as a comma separated file in `bld/bpi_long.csv`.
    
4. We merged the long dataset with the chs dataset. Only keep observations that are present in the chs dataset. We used the columns `childid` and `year` as merge keys and give all variables that appear in both datasets (if there are any) sensible suffixes. Save this as a comma separated file in `bld/bpi_merged.csv`.

5. We calculate scores for each subscale of the bpi by averaging the items of that subscale. The answers 'sometimes true' and 'often true', are counted as 1, 'not true' is counted as zero. Standardize the scores to mean 0 and variance 1 for each age group. We saved the result as a comma separatade file in `bld/bpi_final.csv`.

6. We made regression plots for each subscale that show how your score relates to the same score in the chs data. Note that in the chs data missings are sometimes coded as -100. The names of their score relate to the names as follows: 
    ```python
    {'antisocial': 'bpiA', 'anxiety': 'bpiB', 'headstrong': 'bpiC', 'hyperactive': 'bpiD', 'peer': 'bpiE'}
    ```
    The dependence scale has no counterpart in the chs data. 

    Since the variables are discrete, many datapoints are plotted on top of each other. 
    
   With this step, we got a strong but not perfect negative correlation in each case.

7. The latent factor model behind the technology of skill formation implies that all items that make up the behavior problems index should be positively correlated, with especially high correlations between the items of one subscale. We Made a heatmap of the correlation matrix of the items from at least three subscales and drder the items such that items that belong to the same subscale are closer together. 